In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
import torch.optim as optim
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'torch'

In [2]:
class Net(nn.Module):

    def __init__(self, num_classes=2):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 199) 
        #self.fc2 = nn.Linear(512, 512) 

        self.fc3 = nn.Linear(199, num_classes)

    def forward(self, x):
        x = F.tanh(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = (self.fc3(x))

        return x

model = Net(num_classes = 2)

NameError: name 'nn' is not defined

In [ ]:
mean0 = (1, 1)
cov0 = [[1, 0], [0, 1]]

mean1 = (-1, -1)
cov1 = [[1, 0], [0, 1]]

data0 = np.random.multivariate_normal(mean0, cov0, 1000)
data1 = np.random.multivariate_normal(mean1, cov1, 1000)

dataset = [(torch.Tensor(point), torch.LongTensor([0])) for point in data0]
dataset += [(torch.Tensor(point), torch.LongTensor([1])) for point in data1]

In [ ]:
#uniform noise

mean0 = (1, 1)
sigma = 0.5
rho = sigma * sigma

cov0 = [[rho, 0], [0, rho]]

mean1 = (-1, -1)
cov1 = [[rho, 0], [0, rho]]

data0 = np.random.multivariate_normal(mean0, cov0, 1000)
data1 = np.random.multivariate_normal(mean1, cov1, 1000)

data = np.concatenate((data0, data1))
#data1[:,1] = data1[:,1] - 2
#print(data0)

input = torch.tensor(data).float()


label = [torch.LongTensor(np.random.binomial(1, 0.3, 1)) for point in data0]
label += [torch.LongTensor(np.random.binomial(1, 0.7, 1)) for point in data1]

label = torch.tensor(label)

In [ ]:
#uniform noise

mean0 = (1, 1)
sigma = 0.5
rho = sigma * sigma

cov0 = [[rho, 0], [0, rho]]

mean1 = (-1, -1)
cov1 = [[rho, 0], [0, rho]]

data0 = np.random.multivariate_normal(mean0, cov0, 1000)
data1 = np.random.multivariate_normal(mean1, cov1, 1000)

data = np.concatenate((data0, data1))
#data1[:,1] = data1[:,1] - 2
#print(data0)

input = torch.tensor(data).float()


label = [torch.LongTensor(np.random.binomial(1, 0.3, 1)) for point in data0]
label += [torch.LongTensor(np.random.binomial(1, 0.7, 1)) for point in data1]

label = torch.tensor(label)

In [ ]:
from torch.optim import Optimizer
import math

class Adam(Optimizer):


    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8,
                 weight_decay=0, amsgrad=False):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
        if not 0.0 <= weight_decay:
            raise ValueError("Invalid weight_decay value: {}".format(weight_decay))
        defaults = dict(lr=lr, betas=betas, eps=eps,
                        weight_decay=weight_decay, amsgrad=amsgrad)
        super(Adam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(Adam, self).__setstate__(state)
        for group in self.param_groups:
            group.setdefault('amsgrad', False)

    @torch.no_grad()
    def step(self, closure=None):
        """Performs a single optimization step.

        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad
                if grad.is_sparse:
                    raise RuntimeError('Adam does not support sparse gradients, please consider SparseAdam instead')
                amsgrad = group['amsgrad']

                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    # Exponential moving average of gradient values
                    state['exp_avg'] = torch.zeros_like(p, memory_format=torch.preserve_format)
                    # Exponential moving average of squared gradient values
                    state['exp_avg_sq'] = torch.zeros_like(p, memory_format=torch.preserve_format)
                    if amsgrad:
                        # Maintains max of all exp. moving avg. of sq. grad. values
                        state['max_exp_avg_sq'] = torch.zeros_like(p, memory_format=torch.preserve_format)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                if amsgrad:
                    max_exp_avg_sq = state['max_exp_avg_sq']
                beta1, beta2 = group['betas']

                state['step'] += 1
                bias_correction1 = 1 - beta1 ** state['step']
                bias_correction2 = 1 - beta2 ** state['step']

                if group['weight_decay'] != 0:
                    grad = grad.add(p, alpha=group['weight_decay'])


                # Decay the first and second moment running average coefficient
                exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1)
                exp_avg_sq.mul_(beta2).addcmul_(grad, grad, value=1 - beta2)
                if amsgrad:
                    # Maintains the maximum of all 2nd moment running avg. till now
                    torch.max(max_exp_avg_sq, exp_avg_sq, out=max_exp_avg_sq)
                    # Use the max. for normalizing running avg. of gradient
                    denom = (max_exp_avg_sq.sqrt() / math.sqrt(bias_correction2)).add_(group['eps'])
                else:
                    denom = (exp_avg_sq.sqrt() / math.sqrt(bias_correction2)).add_(group['eps'])

                step_size = group['lr'] / bias_correction1

                p.addcdiv_(exp_avg, denom, value=-step_size)


                update = (exp_avg / denom).view(-1)
                if update.shape[0] > 200:

                #print(update)

                  plt.hist(update.cpu())
                  plt.show()
                  plt.close()
                ###### Zhiyi THIS 'update' is the random variable we care about!

        return loss

In [ ]:
torch.manual_seed(1)



np.random.seed(1)
mean0 = (1, 1)
#sigma = 0.5
rho = sigma * sigma

cov0 = [[rho, 0], [0, rho]]

mean1 = (-1, -1)
cov1 = [[rho, 0], [0, rho]]

data0 = np.random.multivariate_normal(mean0, cov0, 100)
data1 = np.random.multivariate_normal(mean1, cov1, 100)

data = np.concatenate((data0, data1))
#data1[:,1] = data1[:,1] - 2
#print(data0)

input = torch.tensor(data).float().cuda()


label = [torch.LongTensor(np.random.binomial(1, 0.3, 1)) for point in data0]
label += [torch.LongTensor(np.random.binomial(1, 0.7, 1)) for point in data1]

label = torch.tensor(label).cuda()

model1 = Net(num_classes = 2).cuda()
optimizer = Adam(model1.parameters(), lr=1e-3)

model1.train()

#dataset_size = len(label)
#batchsize = 2



for i in range(30):
  print('step: ', i)
  optimizer.zero_grad()
  #batch_ind = np.random.randint(0, dataset_size, batchsize)

  #output = model1(input[batch_ind])
  output = model1(input)
  output = F.log_softmax(output, dim=0)

  #loss = F.nll_loss(output, label[batch_ind])
  loss = F.nll_loss(output, label)
  
  loss.backward()
  optimizer.step()
  if i % 100 == 0:
    #with torch.no_grad:
      #print(loss)
      #output = model1(input)
      #output = F.log_softmax(output, dim=0)
      #loss = F.nll_loss(output, label)
      
      #print(avgpred)
      pred = output.max(dim=1)[1]
      #print(pred == 0)
      avgpred1 = F.softmax(output, dim=1)[pred == 0].mean(dim=0)[0].item()
      avgpred2 = F.softmax(output, dim=1)[pred == 1].mean(dim=0)[1].item()
      avgpred = (avgpred1 + avgpred2)/2

      
      print(loss.item(), (pred == label).sum().float().item() / len(label), avgpred)
#correct[sigma] = (((pred == label).sum().float() / len(label)).item())
#losses[sigma] = loss.item()
#avg_pred[sigma] = avgpred